# Kernels from the DeepDrug3D Open Dataset

We have 589 heme-binding proteins and 1553 nucleotide-binding proteins. Can we predict which one binds which using SOAP descriptors?

Steps:
- Generate a tagged xyz file containing all the structures (might be too big?). Save to intermediates/
- Create a .labels file giving either "heme" or "nucleotide" for each PDB. Save to intermediates/
- (Optional: split the xyz file into chunks for parallelisation)
- Run SOAP++ on the xyz file, using params in input/kernelParams.json
- Read the resulting hdf5 file, cluster, colour according to label in the .labels file, run an SVM.
- Plot the atomic contributions

In [ ]:
import json
import proteinnetworks
import numpy as np
import os
import pandas as pd
import seaborn as sns
import palettable
from scipy.cluster import hierarchy
from collections import defaultdict
from matplotlib.colors import rgb2hex, colorConverter

%matplotlib inline

import sklearn.metrics
from sklearn import svm

import subprocess
import os
from IPython.display import Image
from IPython.display import display


In [ ]:
db = proteinnetworks.database.Database()

In [ ]:
# Convert these to an xyz file, respecting the ordering and saving it.
# (need to use Python2 to get quippy to work)
hemeDirName = "inputs/protein-heme/"
nucleoDirName = "inputs/protein-nucleotide/"
!/home/will/.local/anaconda3/envs/python2env/bin/python generateXYZfile.py {hemeDirName} {nucleoDirName}


In [ ]:
os.listdir(nucleoDirName)

# SOAPXX

Now repeat this analysis with the SOAPXX platform

In [ ]:
import h5py

stub = "dataBinders/tempATPandEstradiol_tagged"

f = h5py.File(f"{stub}.hdf5", "r")
# f = h5py.File("data/tempScop_tagged.hdf5", "r")

soapxxScopData = f['kernel']['kernel_mat'].value
soapxxScopData = np.nan_to_num(soapxxScopData)
f.close()

In [ ]:
with open(f"{tempDirectoryName}.labels") as flines:
    scopPaths = [line.strip().split("/")[1][:-4] for line in flines]

soapxxScopdf = pd.DataFrame(soapxxScopData, columns=scopPaths)
# dictSwap = {i: x for i,x in enumerate(scopPaths)}
# scopdf.rename(index=dictSwap, inplace=True)
soapxxScopdf.head()

In [ ]:
soapxxscopg = sns.clustermap(soapxxScopdf, yticklabels="auto", figsize=(15,15))

In [ ]:
# Well, these aren't the same. What happened?

In [ ]:
d = hierarchy.distance.pdist(soapxxScopdf)
col_linkage = hierarchy.linkage(d, method="average")
R = hierarchy.dendrogram(col_linkage, labels=scopPaths, no_plot=True)


In [ ]:
class Clusters(dict):
    def _repr_html_(self):
        html = '<table style="border: 0;">'
        for c in self:
            hx = rgb2hex(colorConverter.to_rgb(c))
            html += '<tr style="border: 0;">' \
            '<td style="background-color: {0}; ' \
                       'border: 0;">' \
            '<code style="background-color: {0};">'.format(hx)
            html += c + '</code></td>'
            html += '<td style="border: 0"><code>' 
            html += repr(self[c]) + '</code>'
            html += '</td></tr>'
        
        html += '</table>'
        
        return html
    
    
def get_cluster_classes(den, label='ivl'):
    cluster_idxs = defaultdict(list)
    for c, pi in zip(den['color_list'], den['icoord']):
        for leg in pi[1:3]:
            i = (leg - 5.0) / 10.0
            if abs(i - int(i)) < 1e-5:
                cluster_idxs[c].append(int(i))
    
    cluster_classes = Clusters()
    for c, l in cluster_idxs.items():
        i_l = [den[label][i] for i in l]
        cluster_classes[c] = i_l
    
    return cluster_classes

In [ ]:
clusterClasses = get_cluster_classes(R)
clusterClasses

In [ ]:
# Plot the scop classes as colourrows
from palettable.colorbrewer.qualitative import Dark2_6

palette = Dark2_6.hex_colors
scopColours = []
scopClassLabels = []
for path in scopPaths:
    if path.endswith("estradiol"):
        scopColours.append(palette[0])
        scopClassLabels.append("estradiol")
    elif path.endswith("ATP"):
        scopColours.append(palette[1])
        scopClassLabels.append("ATP")
    else:
        raise ValueError

In [ ]:
scopg = sns.clustermap(soapxxScopdf, yticklabels="auto", figsize=(15,15), row_colors=scopColours)
# scopg.savefig("clusteredSimMatrix.png", dpi=300)


# SVM stuff

- Run SVM using this kernel
- Extract the relevant params from the SVM (what they?)
- Get the pij and kij from SOAPXX

Each atom's contribution to the classifer is given by:

\begin{equation*}
\delta_{Z_J, B} = \sum_{A} \alpha_{A} y_{A}  \sum_{i \in A}  P_{ij} k_{ij}(A,B) + \frac{\beta}{|B|}
\end{equation*}

This is the contribution of an individual atomic environment j in structure B to the decision.

- $ \alpha_{A} y_{A}$ are the SVM coefficients, optimised using sklearn.
- $\beta$ is the decision threshold.
- These are extracted from the SVM classifier


- $P_{ij}$ is the permutation matrix mapping enviromments in A to environments in B.
- $k_{ij}(A,B)$ is the SOAP kernel between atomic environments $i \in A$ and $j \in B$
- These are extracted from SOAPXX




In [ ]:
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt\\\
# import palettable
# import json
import sklearn.metrics
from sklearn import svm


def shuffledCopies(a, b):
    """Shuffled a and b, where a is 1d and b is 2d symmetric"""
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    shuffled = b[p]
    b = shuffled[:,p]
    return a[p], b

def getSVMScores(path, labels):
    """
    Given a path to a *.k glosim coefficient matrix, and the training
    labels, run kernel SVM and print the classification report.
    """
    with open(path) as flines:
        glosimData = [line.strip() for line in flines][1:]
    glosimData = np.asarray([line.split() for line in glosimData], dtype=float)
    # strip nans and infs (sketchy)
    glosimData = np.nan_to_num(glosimData)
    glosimData[np.where(glosimData>1)] = 1
    glosimData[np.where(glosimData<0)] = 0

    # Shuffle both the labels and the testMatrix
    avgF1Score = []
    numberOfTrials = 1000
    for i in range(numberOfTrials):
        labels, glosimData = shuffledCopies(np.asarray(labels),glosimData)

        trainSize = int(len(glosimData)*2/3)
        x_train = glosimData[:trainSize, :trainSize]
        x_test = glosimData[trainSize:, :trainSize]
        y_train = labels[:trainSize]
        y_test = labels[trainSize:]
        clf = svm.SVC(kernel="precomputed", verbose=False, max_iter=1e9, C=1)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        scores = sklearn.metrics.classification_report(y_test, y_pred, output_dict=True)
        f1 = scores["weighted avg"]["f1-score"]
        avgF1Score.append(f1)
    return sum(avgF1Score)/len(avgF1Score)

In [ ]:
kernelLabels, kernelData = shuffledCopies(np.asarray(scopClassLabels), soapxxScopData)
# kernelLabels, kernelData = (np.asarray(scopClassLabels), soapxxScopData)


In [ ]:
trainSize = int(len(kernelData)*2/3)
x_train = kernelData[:trainSize, :trainSize]
x_test = kernelData[trainSize:, :trainSize]
y_train = kernelLabels[:trainSize]
y_test = kernelLabels[trainSize:]
clf = svm.SVC(kernel="precomputed", verbose=False, max_iter=1e9, C=1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
scores = sklearn.metrics.classification_report(y_test, y_pred) #  output_dict=True)
print(scores)


# Pymol Viz


This all needs pipelining properly, but now we have the dzs giving the contribution of each residue to the SVM. 

Carl makes a density field $ \rho_B(r) = \sum_{j \in B} \delta_{z_j, B} N \left( r_j, \sigma_j \right)$, i.e a bunch of atom-centred Gaussians of width $\sigma = 0.5 A$. 

I think I'll just colour residues using the b-factor then spectrum it.

In [ ]:
def getAtomicContributionGivenAtomAndStructure(clf, atomIndex, structureIndex):
    """
    Given a atom index j and a structure index B, get the contribution to the  decision function.
    
    \delta_{Z_J, B} = \sum_{A} \alpha_{A} y_{A}  \sum_{i \in A}  P_{ij} k_{ij}(A,B) + \frac{\beta}{|B|}
    """
    
    dz = 0.0
    # Loop over structures A 
    for dualcoef, supportIndex in zip(clf.dual_coef_[0], clf.support_):
        # Which way round should these indices be?
        p_base = np.load(f"dataBinders/tempATPandEstradiol_tagged_perms_{structureIndex}_{supportIndex}.dat.npy")
        k_base = np.load(f"dataBinders/tempATPandEstradiol_tagged_kerns_{structureIndex}_{supportIndex}.dat.npy")
        # Loop over each atom i in structure A       
        p_base_row = p_base[atomIndex] # should be a row of length (atoms in supportIndex)
        k_base_row = k_base[atomIndex] # should be a row of length (atoms in supportIndex)
        dz += dualcoef*np.sum(p_base_row*k_base_row)
        
    sizeOfB = len(p_base) 
    dz += clf.intercept_ /sizeOfB
    return dz


In [ ]:
def plotAtomicContributionsGivenDeltaZMappingAndPDBRef(deltaZmapping, pdbRef):
    """
    Given the PDB reference, chain reference and a list of (residueNumber, deltaZ) tuples,
    Plot the deltaZs onto the structure.
    """
    PDBData = db.extractPDBFile(pdbRef)

    data = np.asarray([line.strip() for line in PDBData
                       if line[:4] == "ATOM"
#                        and line[21] == chainRef
                      ])
    with open("temp.pdb", "w") as flines:
        flines.write("\n".join(data))

    dzs = [x[1] for x in deltaZmapping]
    
    # Make the spectrum symmetric
    lowestValue = min(x[0] for x in dzs)
    highestValue = max(x[0] for x in dzs)
    if lowestValue < 0 and highestValue > 0:
        if abs(lowestValue) > highestValue:
            highestValue = - lowestValue
        elif abs(lowestValue) < highestValue:
            lowestValue = -highestValue    

    pymolScript = f"load temp.pdb, {pdbRef}\n"
    pymolScript += f"alter {pdbRef}, b=-1\n"

    for resi,dz in deltaZmapping: # might not work if the residue ids are off
        pymolScript += f"alter resi {resi}, b={dz[0]}\n"

    pymolScript += f"""
    #formatting
    bg_color white
    hide all
    #show sticks
    show cartoon
    spectrum b, blue_white_red, minimum={lowestValue}, maximum={highestValue}
    set opaque_background=0
    set antialias = on
    set line_smooth = 1
    set depth_cue = 1
    set specular = 1
    set surface_quality = 1
    set stick_quality = 15
    set sphere_quality = 2
    set ray_trace_fog = 0.8
    set light = (-0.2,0,-1)

    set ray_shadows, 0
    set surface_mode, 1
    set cartoon_side_chain_helper,on
    zoom
    rebuild
    """
    pymolScript += f"save {pdbRef}.pse \n"
    pymolScript += f"""
    set ray_trace_mode = 1
    png {pdbRef}.png, width=10cm, dpi=300, ray=1
    """

    with open("temp.pml", mode='w') as flines:
        flines.write(pymolScript)

    # Run quietly
    subprocess.run(["pymol", "-c", "temp.pml"])
    os.remove("temp.pml")
    os.remove("temp.pdb")
    display(Image(f"{pdbRef}.png"))
#     os.remove(f"{pdbRef}.png")


In [ ]:
def plotAtomicContributionsGivenIndexOnSCOPPaths(indexNumber):
    """
    Given an index to SCOPPaths (the list of what label corresponds to what row of the kernel matrix),
    find the atomic contributions, then plot them.
    """
    global scopPaths
    
    pdbRef, chainRef, *_ = scopPaths[indexNumber].split("_")
    print(pdbRef, chainRef)
    data = !grep {scopPaths[indexNumber]}   dataBinders/tempATPandEstradiol_tagged.xyz -B 1
    sizeOfStructure = int(data[0])
    dzs= []
    for atomIndex in range(sizeOfStructure):
        dz = getAtomicContributionGivenAtomAndStructure(clf, atomIndex,indexNumber)
        dzs.append(dz)
    with open(f"tempATPandEstradiol/{scopPaths[indexNumber]}.pdb") as flines:
        data = flines.readlines()
    residueSequenceNumbers = []
    for line in data:
        residueSequenceNumbers.append(line[22:26].strip())
        
        
    plotAtomicContributionsGivenDeltaZMappingAndPDBRef(list(zip(residueSequenceNumbers,dzs)), pdbRef)


In [ ]:
for i, path in enumerate(scopPaths):
    pdbRef, *_ = scopPaths[i].split("_")
    plotAtomicContributionsGivenIndexOnSCOPPaths(i)
